## Импортирование и настройка необходимых библиотек

In [71]:
import pandas as pd
import mglearn
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

## Считывание датасета

In [72]:
df = pd.read_csv(r'C:\Users\Rasim\Desktop\DataAnalysis\DataSets\ORGANICS.csv')
display(df)
print(df.columns)

,ID,DemAffl,DemAge,DemCluster,DemClusterGroup,DemGender,DemReg,DemTVReg,PromClass,PromSpend,PromTime,TargetBuy,TargetAmt
0,140,10.0,76.0,16.0,C,U,Midlands,Wales & West,Gold,16000.00,4.0,0,0
1,620,4.0,49.0,35.0,D,U,Midlands,Wales & West,Gold,6000.00,5.0,0,0
2,868,5.0,70.0,27.0,D,F,Midlands,Wales & West,Silver,0.02,8.0,1,1
3,1120,10.0,65.0,51.0,F,M,Midlands,Midlands,Tin,0.01,7.0,1,1
4,2313,11.0,68.0,4.0,A,F,Midlands,Midlands,Tin,0.01,8.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22218,52834058,13.0,65.0,9.0,B,F,South East,London,Silver,1500.00,5.0,0,0
22219,52834376,15.0,73.0,34.0,D,U,South East,S & S East,Gold,6053.06,12.0,0,0
22220,52837057,9.0,70.0,15.0,B,F,North,Yorkshire,Gold,6000.00,5.0,0,0
22221,52838096,11.0,66.0,8.0,B,F,North,N West,Silver,5000.00,5.0,0,0


Index(['ID', 'DemAffl', 'DemAge', 'DemCluster', 'DemClusterGroup', 'DemGender',
       'DemReg', 'DemTVReg', 'PromClass', 'PromSpend', 'PromTime', 'TargetBuy',
       'TargetAmt'],
      dtype='object')


## Препроцессинг данных

### Удаление бесполезных колонок

In [73]:
data = df.drop(columns=['ID', 'TargetAmt'])
display(data)

,DemAffl,DemAge,DemCluster,DemClusterGroup,DemGender,DemReg,DemTVReg,PromClass,PromSpend,PromTime,TargetBuy
0,10.0,76.0,16.0,C,U,Midlands,Wales & West,Gold,16000.00,4.0,0
1,4.0,49.0,35.0,D,U,Midlands,Wales & West,Gold,6000.00,5.0,0
2,5.0,70.0,27.0,D,F,Midlands,Wales & West,Silver,0.02,8.0,1
3,10.0,65.0,51.0,F,M,Midlands,Midlands,Tin,0.01,7.0,1
4,11.0,68.0,4.0,A,F,Midlands,Midlands,Tin,0.01,8.0,0
...,...,...,...,...,...,...,...,...,...,...,...
22218,13.0,65.0,9.0,B,F,South East,London,Silver,1500.00,5.0,0
22219,15.0,73.0,34.0,D,U,South East,S & S East,Gold,6053.06,12.0,0
22220,9.0,70.0,15.0,B,F,North,Yorkshire,Gold,6000.00,5.0,0
22221,11.0,66.0,8.0,B,F,North,N West,Silver,5000.00,5.0,0


### Обработка пропущенных значений

In [74]:
print(data.isnull().sum())

DemAffl            1085
DemAge             1508
DemCluster          674
DemClusterGroup     674
DemGender          2512
DemReg              465
DemTVReg            465
PromClass             0
PromSpend             0
PromTime            281
TargetBuy             0
dtype: int64


Заменим пропущенные числовые данные на их медиану

In [75]:
data['DemAffl'].fillna(data['DemAffl'].median(), inplace=True)
data['DemAge'].fillna(data['DemAge'].median(), inplace=True)
data['DemCluster'].fillna(data['DemCluster'].median(), inplace=True)
data['PromTime'].fillna(data['PromTime'].median(), inplace=True)

Заменим пропущенные категориальные данные на их моду

In [76]:
data['DemClusterGroup'].fillna(data['DemClusterGroup'].mode()[0], inplace=True)
data['DemGender'].fillna(data['DemGender'].mode()[0], inplace=True)
data['DemTVReg'].fillna(data['DemTVReg'].mode()[0], inplace=True)
data['DemReg'].fillna(data['DemReg'].mode()[0], inplace=True)
display(data)

,DemAffl,DemAge,DemCluster,DemClusterGroup,DemGender,DemReg,DemTVReg,PromClass,PromSpend,PromTime,TargetBuy
0,10.0,76.0,16.0,C,U,Midlands,Wales & West,Gold,16000.00,4.0,0
1,4.0,49.0,35.0,D,U,Midlands,Wales & West,Gold,6000.00,5.0,0
2,5.0,70.0,27.0,D,F,Midlands,Wales & West,Silver,0.02,8.0,1
3,10.0,65.0,51.0,F,M,Midlands,Midlands,Tin,0.01,7.0,1
4,11.0,68.0,4.0,A,F,Midlands,Midlands,Tin,0.01,8.0,0
...,...,...,...,...,...,...,...,...,...,...,...
22218,13.0,65.0,9.0,B,F,South East,London,Silver,1500.00,5.0,0
22219,15.0,73.0,34.0,D,U,South East,S & S East,Gold,6053.06,12.0,0
22220,9.0,70.0,15.0,B,F,North,Yorkshire,Gold,6000.00,5.0,0
22221,11.0,66.0,8.0,B,F,North,N West,Silver,5000.00,5.0,0


In [77]:
print(data.isnull().sum())

DemAffl            0
DemAge             0
DemCluster         0
DemClusterGroup    0
DemGender          0
DemReg             0
DemTVReg           0
PromClass          0
PromSpend          0
PromTime           0
TargetBuy          0
dtype: int64


Вроде избавились от пропущенных значений, однако заметим что в колонке Gender одно из значений U - unknown, что является грубо говоря пропущенным значением. Но я хочу рассмотреть это как третью группу людей: те, кто не захотел указывать пол.

### Даммирование категориальных признаков

In [78]:
dummy = pd.get_dummies(data)
#Перенос колонки TargetBuy в конец таблицы
dummy = dummy[[column for column in dummy if column != 'TargetBuy'] + ['TargetBuy']]
display(dummy)

,DemAffl,DemAge,DemCluster,PromSpend,PromTime,DemClusterGroup_A,DemClusterGroup_B,DemClusterGroup_C,DemClusterGroup_D,DemClusterGroup_E,...,DemTVReg_S & S East,DemTVReg_S West,DemTVReg_Ulster,DemTVReg_Wales & West,DemTVReg_Yorkshire,PromClass_Gold,PromClass_Platinum,PromClass_Silver,PromClass_Tin,TargetBuy
0,10.0,76.0,16.0,16000.00,4.0,0,0,1,0,0,...,0,0,0,1,0,1,0,0,0,0
1,4.0,49.0,35.0,6000.00,5.0,0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,0
2,5.0,70.0,27.0,0.02,8.0,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,1
3,10.0,65.0,51.0,0.01,7.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
4,11.0,68.0,4.0,0.01,8.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22218,13.0,65.0,9.0,1500.00,5.0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
22219,15.0,73.0,34.0,6053.06,12.0,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,0
22220,9.0,70.0,15.0,6000.00,5.0,0,1,0,0,0,...,0,0,0,0,1,1,0,0,0,0
22221,11.0,66.0,8.0,5000.00,5.0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [79]:
X = dummy.iloc[:, :-1].values
y = dummy['TargetBuy'].values
display(x.shape, y.shape)

(22223, 37)

(22223,)

## Оценка и улучшение качества модели

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
tree = DecisionTreeClassifier(max_depth=35).fit(X_train, y_train)

Найдем долю правильных ответов полученного алгоритма на обучающей выборке

In [81]:
y_pred_train = tree.predict(X_train)
round(accuracy_score(y_pred_train, y_train), 2)

1.0

Теперь найдите долю правильных ответов полученного алгоритма на тестовой выборке

In [82]:
y_pred = tree.predict(X_test)
round(accuracy_score(y_pred, y_test), 2)

0.71

### Перекрестная проверка

In [83]:
scores = cross_val_score(tree, X, y, cv=5)
print("Значения правильности перекрестной проверки:\n {}".format(scores))
print()
print("Средняя правильность перекрестной проверки: {:.2f}".format(scores.mean()))

Значения правильности перекрестной проверки:
 [0.71698538 0.72913386 0.71271091 0.71377138 0.71849685]

Средняя правильность перекрестной проверки: 0.72


### Стратифицированная k-блочная перекрестная проверка

In [84]:
kfold5 = KFold(n_splits=5)
print("Значения правильности перекрестной проверки:\n{}".format(
      cross_val_score(tree, X, y, cv=kfold5)))

Значения правильности перекрестной проверки:
[0.71698538 0.72260967 0.71856018 0.7180468  0.71422142]


In [85]:
kfold3 = KFold(n_splits=3)
print("Значения правильности перекрестной проверки:\n{}".format(
      cross_val_score(tree, X, y, cv=kfold3)))

Значения правильности перекрестной проверки:
[0.71679266 0.71719762 0.71351424]


In [86]:
kfold = KFold(n_splits=5, shuffle=True, random_state=0)
print("Значения правильности перекрестной проверки:\n{}".format(
    cross_val_score(tree, X, y, cv=kfold)))

Значения правильности перекрестной проверки:
[0.72418448 0.71158605 0.71091114 0.7119712  0.71084608]


### Перекрестная проверка с исключением по одному

In [87]:
loo = LeaveOneOut()
scores = cross_val_score(tree, X, y, cv=loo)
print("Количество итераций cv: ", len(scores))
print("Средняя правильность: {:.2f}".format(scores.mean()))

Количество итераций cv:  22223
Средняя правильность: 0.72


### Перекрестная проверка со случайными перестановками при разбиении

In [89]:
shuffle_split = ShuffleSplit(test_size=.5, train_size=.5, n_splits=10)
scores = cross_val_score(tree, X, y, cv=shuffle_split)
print("Значения правильности перекрестной проверки:\n{}".format(scores))

Значения правильности перекрестной проверки:
[0.71409287 0.70968323 0.71184305 0.72156228 0.71337293 0.71238301
 0.71346292 0.71787257 0.70887329 0.71328294]


### Подбор оптимальных параметров

In [90]:
params = {'max_depth' : [3, 4, 5, 6, 7, 8, 9, 10, None],
                                'max_features' : ['auto', 'log2', None],
                                'min_samples_leaf' : range(1, 10),
                                'min_samples_split' : range(2, 10),
                                'criterion' : ['gini', 'entropy']}
gs = GridSearchCV(DecisionTreeClassifier(random_state=322), param_grid = params, cv=5, scoring='accuracy')
gs.fit(X_train, y_train)
print(gs.best_params_)
print('Лучший результат (accuracy):', gs.best_score_)

{'criterion': 'entropy', 'max_depth': 5, 'max_features': None, 'min_samples_leaf': 2, 'min_samples_split': 2}
Лучший результат (accuracy): 0.8060238795224095
